# Tutorial for identifying which ROIs extracted by suite3d are "red"
Or otherwise express a structural fluorophore measured on a different channel.

## Overview
In neuroscience, we often express multiple fluorophores in cells, in which one is used for measuring activity - the "functional" fluorophore - and one is used for labeling the cell - the "structural" fluorophore. In this case, we usually want to know which ROIs express the structural fluorophore. Suite3d enables this by providing a way to compute a registered mean fluorescence image for the structural channel. Then, the package called "cellector" can compare the volumetric ROIs and the structural fluorescence to identify which ROIs have this second fluorophore (usually something like tdTomato).

## Steps
1. Set params to specify which channel is structural and that you want to process it. 
2. Run suite3d pipeline normally -- everything is handled automatically.
3. Process results in cellector. 

In [ ]:
import os 
from datetime import date
from matplotlib import pyplot as plt
import numpy as np

# we need to set the current path to the directory
# containing the suite3d repository, this hack should
# do the trick
os.chdir(os.path.dirname(os.path.abspath("")))

from suite3d.job import Job
from suite3d import io
from suite3d import plot_utils as plot

In [ ]:
# update this to point to the demo data! 
tifs = io.get_tif_paths('/mnt/md0/data/demo/lbm')[:10]
for tif in tifs: print(tif)

In [3]:
# update this to point to the demo data! 
tifs = io.get_tif_paths('/mnt/md0/data/demo/standard-2p')[:10]

## Special Parameters for Processing Structural Fluorescence
The params dictionary is where you (the user) tell suite3d how to process your imaging data. There are three parameters relating to structural fluorescence processing and red cell identification. 

> **Note on terminology**
>
> Suite3d makes a distinction between the "functional" and "structural" channels. The functional channel is the one that is used for the primary analysis (e.g. Ca2+ imaging). The structural channel is the one that is used for identifying which ROIs have the structural fluorophore. 
>
> The functional channel is used to register the data, segment the ROIs, and extract dynamic fluorescence traces. The structural channel is simply registered and used to compute a mean image that is co-registered with the functional channel.
> 
> So when we say "red cell identification", we mean identifying any ROIs that match fluorescence features in the "structural" channel (which is usually red, but of course could be any color). 

1. `process_structural_channel`: whether to process a second fluorescence channel representing structural information (like tdTomato, for example)
    - this is a boolean `True` or `False` and is set to `False` by default
2. `structural_color_channel`: the color channel that represents the structural fluorophore
    - this is an integer that corresponds to the color channel in the tif file. Suite3d expects that your tif files are shaped such that each frame is 4D with shape (frames, colors, y-pixels, x-pixels). Remember python indexing, so the first color channel is 0.
3. `clear_registered_structural_data`: whether to clear the registered structural data after the mean volume is calculated
    - this is a boolean `True` or `False` and is set to `False` by default.
    - depending on your imaging system, the registered structural data may take up a lot of memory. The only reasons to set this to `True` is if you want to use the structural channel for something like analysis of how suite3d performed the registration. 

Putting it all together, a standard params for red cell identification might look like this:
> ```python
> index_to_structural_channel = 1 # which color channel is the structural one
> params = { 
>     # Parameters for processing structural fluorescence    
>     'process_structural_channel': True, # whether to process the structural channel
>     'structural_color_channel': index_to_structural_channel, # which color channel is the structural one
>     'clear_registered_structural_data': False, # whether to clear the registered structural data after the mean volume is calculated
> }
> ```

In [4]:
# Set the mandatory parameters
params = {
    # volume rate
    'fs': io.get_vol_rate(tifs[0]),
    
    # planes to analyze. 0 is typically the flyback, so we exclude it here
    'planes' : np.array([1,2,3,4]), 
    # number of planes recorded by scanimage, including the flyback
    'n_ch_tif' : 5,
    
    # Decay time of the Ca indicator in seconds. 1.3 for GCaMP6s. This example is for GCamP8m
    'tau' : 1.3,
    'lbm' : False, 
    'num_colors' : 2, # how many color channels were recorded by scanimage
    'functional_color_channel' : 0, # which color channel is the functional one

    # Parameters for processing structural fluorescence    
    'process_structural_channel': True, # whether to process the structural channel
    'structural_color_channel': 1, # which color channel is the structural one
    'clear_registered_structural_data': False, # whether to clear the registered structural data after the mean volume is calculated

     # voxel size in z,y,x in microns
    'voxel_size_um' : (20, 1.5, 1.5),

    # number of files to use for the initial pass
    # usually, ~500 frames is a good rule of thumb
    # we will just use 200 here for speed
    'n_init_files' :  2,

    # 3D GPU registration - fast! 
    '3d_reg' : True,
    'gpu_reg' : True,
    
    # note : 3D CPU is not supported yet
    'subtract_crosstalk' : False, # turn off some lbm-only features
    'fuse_strips' : False, # turn off some lbm-only features
}

In [23]:
# Create the job
job = Job(r'/mnt/md0/runs','demo-std', tifs = tifs,
          params=params, create=True, overwrite=True, verbosity = 3)

In [22]:
job.run_init_pass()

In [9]:
# If you have large tiffs, split the large tiffs into files of size 100 after registration
job.params['split_tif_size'] = 100

In [11]:
# OPTIONAL: load and take a look at the reference image
summary = job.load_summary()
ref_img = summary['ref_img_3d']

In [21]:
# Register will register the structural fluorescence using the same 
# values as for functional to ensure they overlap perfectly. It will
# produce a registered movie and a registered mean image. 
job.register()

In [20]:
corr_map = job.calculate_corr_map()

In [14]:
res = job.load_corr_map_results()
vmap = res['vmap']

In [18]:
# for speed, only segment a single patch
job.params['patch_size_xy'] = (550, 550)
job.segment_rois()

In [ ]:
job.compute_npil_masks()
traces = job.extract_and_deconvolve()

In [ ]:
job.export_results('path/to/output', result_dir_name='rois')

## Curation of functional ROIs

To take a look at the outputs in napari, navigate to the suite3d directory in a command shell and run the following:
```
python curation.py curation --output_dir /path/to/output/rois
```

## Processing with cellector

Cellector is a package that can be used to process the results of suite3d and identify which ROIs have the structural fluorophore. 

If you haven't installed cellector yet, you can do so by running the following code in while in your suite3d environment:
```
pip install cellector
```
(Suite3d includes this by default now, but it didn't use to so you may need to install it separately.)


For more info on how to use cellector, see the [cellector documentation](https://cellector.readthedocs.io/en/stable/).


#### Using the GUI
Once you have opened the GUI, you can set the criteria cutoffs for determining which ROIs are red (or another color). Then, save the results (there's a button to click or you can use ctrl-s).

Once you've saved the results -- you can find the cell selection in the 'path/to/output/folder_name/cellector/' directory with the filename:

"idx_selected.npy"


In [1]:
from cellector.io import create_from_suite3d
from cellector.gui import SelectionGUI

suite3d_dir = 'path/to/output/folder_name'
roi_processor = create_from_suite3d(suite3d_dir)
gui = SelectionGUI(roi_processor)